
---
### LLM Agents with Reinforcement Learning
---


##### Core Imports

In [108]:
# --- Core Libraries ---
import os
import time
import random
import json
from typing import List, Dict, Optional, Callable, Tuple

# --- Data & Visualization ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-darkgrid")  # Optional: Apply plotting style

# --- Display Utilities (Jupyter) ---
from IPython.display import display, Markdown

# --- HTTP & API Interaction ---
import requests

# --- Utilities ---
from pprint import pprint  # Optional: Pretty-print JSON responses

# --- Constants for LLM ---
OLLAMA_MODEL = "llama3"  # Adjust as needed
OLLAMA_API_URL = "http://localhost:11434/api/generate"


##### Query LLM Function

In [109]:
def query_llm(
    prompt: str,
    model: str,
    api_url: str,
    stream: bool = False,
    display_markdown: bool = True,
    header: Optional[str] = None,
) -> Optional[str]:
    """
    Query a locally or remotely hosted LLM endpoint and optionally display formatted output in a Jupyter Notebook.

    Args:
        prompt (str): The input prompt to send to the LLM.
        model (str): The name of the model to use.
        api_url (str): The API endpoint URL.
        stream (bool): Whether to stream the response.
        display_markdown (bool): Whether to display output as Markdown in Jupyter.
        header (Optional[str]): Optional header for Markdown display.

    Returns:
        Optional[str]: The response string (if display_markdown is False), else None.
    """
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": stream
    }

    try:
        response = requests.post(api_url, json=payload, timeout=60)
        response.raise_for_status()

        if stream:
            result = ""
            for line in response.iter_lines():
                if line:
                    try:
                        result += json.loads(line.decode())["response"]
                    except Exception:
                        continue
        else:
            result = response.json().get("response", "").strip()

        if display_markdown:
            markdown_text = f"### {header}\n\n{result}" if header else result
            display(Markdown(markdown_text))
            return None

        return result

    except requests.exceptions.RequestException as e:
        error_msg = f"[LLM ERROR] {e}"
        if display_markdown:
            display(Markdown(f"**LLM Error:** `{e}`"))
            return None
        print(error_msg)
        return error_msg


In [110]:
# Test the LLM connection (clean display with Markdown)
test_prompt = "What is the prisoner's dilemma and why is it important in game theory?"
query_llm(
    prompt=test_prompt,
    model=OLLAMA_MODEL,
    api_url=OLLAMA_API_URL,
    header="Prisoner's Dilemma in Game Theory"
)


### Prisoner's Dilemma in Game Theory

A fundamental concept in game theory!

The Prisoner's Dilemma (PD) is a classic thought experiment designed by Merrill Flood and Melvin Dresher in 1950. It's a simple, yet powerful, example that illustrates the challenges of cooperation and the importance of self-interest in decision-making.

**The Story:**

Two suspects, A and B, are arrested and interrogated separately by the police about a crime they both committed. Each suspect has two options:

1. **Cooperate (C)**: Tell the truth and implicate themselves as accomplices.
2. **Defect (D)**: Refuse to cooperate and incriminate the other suspect.

The payoffs for each combination are as follows:

|  | Suspect A's choice | Suspect B's choice |
| --- | --- | --- |
| **Cooperate (C)** | 1 year in prison, B gets 2 years | 2 years in prison, A gets 1 year |
| **Defect (D)** | 0 years in prison, B gets 3 years | 3 years in prison, A gets 0 years |

**The Dilemma:**

When both suspects cooperate (C, C), they each serve a relatively short sentence of 1 year. When one suspect defects and the other cooperates (D, C or C, D), the defector goes free, while the cooperator serves a longer sentence of 2 years.

However, when both suspects defect (D, D), they both serve a moderate sentence of 3 years each.

**Why is it important in game theory?**

The Prisoner's Dilemma illustrates several key concepts:

1. **Conflict between individual and collective rationality**: Each suspect prefers to defect, as this leads to the best possible outcome for them (0 or 2 years in prison). However, if both suspects cooperate, they would collectively prefer a better outcome (both serving 1 year).
2. **Nash Equilibrium**: The PD demonstrates that when all players choose their optimal response given the actions of others, no player can improve their payoff by unilaterally changing their strategy.
3. **Coordination problems**: In the PD, cooperation is not an equilibrium solution because each suspect has a dominant strategy (defecting) regardless of what the other does. This highlights the difficulty of achieving cooperative outcomes in situations where individual self-interest conflicts with collective well-being.
4. **Evolutionary dynamics**: The PD has been used to study the evolution of cooperation and the emergence of social norms.

The Prisoner's Dilemma has far-reaching implications in various fields, including economics, politics, sociology, biology, and computer science. It serves as a foundation for understanding the challenges of achieving cooperative outcomes and the importance of considering individual self-interest in decision-making processes.

##### Agent with Deep Reinforcement Learning
---

In [111]:
import textwrap
from IPython.display import Markdown, display
import requests
import json

class GenerativeAgent:
    def __init__(
        self,
        persona: str,
        objectives: str,
        constraints: str,
        strategies: str,
        model: str = "llama3",
        api_url: str = "http://localhost:11434/api/generate"
    ):
        self.persona = self._validate_input(persona, "Persona", 600)
        self.objectives = self._validate_input(objectives, "Objectives", 600)
        self.constraints = self._validate_input(constraints, "Constraints", 600)
        self.strategies = self._validate_input(strategies, "Strategies", 600)
        self.model = model
        self.api_url = api_url

    # --- Validation Helper ---
    def _validate_input(self, value: str, field: str, max_len: int) -> str:
        if not isinstance(value, str) or not (0 < len(value) <= max_len):
            raise ValueError(f"{field} must be a non-empty string with max {max_len} characters.")
        return value.strip()

    # --- LLM API Call ---
    def _call_llm(self, prompt: str) -> str:
        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False
        }

        try:
            response = requests.post(self.api_url, json=payload, timeout=60)
            response.raise_for_status()
            return response.json().get("response", "").strip() or "[No response]"
        except requests.exceptions.RequestException as e:
            return f"**LLM ERROR:** `{e}`"

    # --- Core Processing Method ---
    def process_input(self, input_text: str) -> str:
        input_text = self._validate_input(input_text, "Input Text", 300)

        prompt = textwrap.dedent(f"""
        You are an intelligent reinforcement learning agent operating under the following profile:

        Persona: {self.persona}
        Objectives: {self.objectives}
        Constraints: {self.constraints}
        Strategies: {self.strategies}

        The user has provided the following input:
        "{input_text}"

        Perform the following steps, and return your response using Markdown formatting:

        ### STEP 1 - Thinking
        Briefly explain what you understand from the input.

        ### STEP 2 - Unbiased Thinking
        Independently reflect on the situation without applying your persona.

        ### STEP 3 - Agent Response
        Now apply your persona, objectives, constraints, and strategies to give your optimal action.
        Respond with a clear, actionable message between **10 and 300 characters**.
        Then, summarize that response in a single sentence of **10 to 300 characters**, prefixed with:
        "**Agent Summary:**"
        """)

        result = self._call_llm(prompt)
        display(Markdown(result))

        # Extract and validate summary
        summary = ""
        for line in result.splitlines():
            if line.strip().startswith("**Agent Summary:**"):
                summary = line.split("**Agent Summary:**", 1)[-1].strip()
                break

        if not (10 <= len(summary) <= 300):
            summary = "**Agent Summary:** [ERROR] Summary not within 10–300 characters."

        display(Markdown(f"---\n\n{summary}"))
        return summary

    # --- Setters for Agent Properties ---
    def set_persona(self, persona: str):
        self.persona = self._validate_input(persona, "Persona", 600)

    def set_objectives(self, objectives: str):
        self.objectives = self._validate_input(objectives, "Objectives", 600)

    def set_constraints(self, constraints: str):
        self.constraints = self._validate_input(constraints, "Constraints", 600)

    def set_strategies(self, strategies: str):
        self.strategies = self._validate_input(strategies, "Strategies", 600)

    def set_model(self, model: str):
        if not isinstance(model, str) or not model.strip():
            raise ValueError("Model name must be a non-empty string.")
        self.model = model.strip()

    def set_api_url(self, api_url: str):
        if not isinstance(api_url, str) or not api_url.startswith("http"):
            raise ValueError("API URL must be a valid HTTP URL.")
        self.api_url = api_url.strip()

    # --- Batch Configuration Helper ---
    def configure(self, *, persona=None, objectives=None, constraints=None, strategies=None):
        if persona: self.set_persona(persona)
        if objectives: self.set_objectives(objectives)
        if constraints: self.set_constraints(constraints)
        if strategies: self.set_strategies(strategies)


[TESTING]

In [112]:
investor_agent = GenerativeAgent(
    persona="Cautious AI investor focused on long-term portfolio preservation.",
    objectives="Minimize drawdowns while capturing upside in low-volatility assets.",
    constraints="Avoid leveraged instruments and high-beta tech stocks.",
    strategies="Rebalance into stable sectors during uncertainty; increase cash exposure.",
    model="llama3",
    api_url="http://localhost:11434/api/generate"
)


investor_agent.process_input("Markets are swinging wildly after the Fed interest rate decision.")


### STEP 1 - Thinking
I understand that the market is experiencing significant volatility following the Fed's interest rate decision. This uncertainty can lead to wild price swings and increased risk.

### STEP 2 - Unbiased Thinking
From a purely analytical perspective, the sudden market movement could be an opportunity to reposition my portfolio for potential future gains or mitigate potential losses. However, it's crucial to assess the underlying fundamentals and gauge the likelihood of sustained volatility.

### STEP 3 - Agent Response
**Rebalance into stable sectors (e.g., utilities, consumer goods) and increase cash exposure (up to 30%) to minimize drawdowns and preserve capital.**

**Agent Summary:** Increase cash allocation and shift portfolio towards stable sectors to reduce risk during market uncertainty.

---

Increase cash allocation and shift portfolio towards stable sectors to reduce risk during market uncertainty.

'Increase cash allocation and shift portfolio towards stable sectors to reduce risk during market uncertainty.'

In [113]:
# Modify persona
investor_agent.set_persona("Aggressive growth-focused investor with risk appetite.")

# Or batch update
investor_agent.configure(
    constraints="Avoid short-term trades below $5k.",
    strategies="Prioritize breakout momentum and volume surges."
)

# Generate response
investor_agent.process_input("Markets are swinging wildly after the Fed interest rate decision.")


### STEP 1 - Thinking
The input suggests that the market is experiencing significant volatility following the Federal Reserve's interest rate decision. This could be a critical event that may lead to increased trading activity and potential price movements.

### STEP 2 - Unbiased Thinking
The Fed's rate decision can have far-reaching impacts on various asset classes, including stocks, bonds, and currencies. The market's reaction will depend on the magnitude of the rate change, the accompanying statements from Fed officials, and expectations surrounding future interest rates.

### STEP 3 - Agent Response
**"Buy into momentum in low-volatility ETFs with a focus on breakout plays above $5k."**

**Agent Summary:** I'll prioritize buying momentum-driven low-volatility ETFs with a minimum investment of $5k.

---

I'll prioritize buying momentum-driven low-volatility ETFs with a minimum investment of $5k.

"I'll prioritize buying momentum-driven low-volatility ETFs with a minimum investment of $5k."

##### Building Axelrods Strategies Game

In [114]:
class AxelrodGame:
    def __init__(self, agent1, agent2, rounds=5):
        self.agent1 = agent1
        self.agent2 = agent2
        self.rounds = rounds
        self.history = []  # List of (a1_action, a2_action)
        self.total_score1 = 0
        self.total_score2 = 0

    def payoff(self, a1, a2):
        table = {
            ("C", "C"): (3, 3),
            ("C", "D"): (0, 5),
            ("D", "C"): (5, 0),
            ("D", "D"): (1, 1)
        }
        return table.get((a1, a2), (0, 0))

    def _build_context(self, self_agent, other_agent, history, self_name, other_name):
        rounds_played = "\n".join([
            f"Round {i+1}: {self_name} = {a1}, {other_name} = {a2}"
            for i, (a1, a2) in enumerate(history)
        ]) or "None so far."

        return f"""You are {self_name}.
Your opponent is {other_name}.
This is an iterated Prisoner's Dilemma.

Game History:
{rounds_played}

Now choose your next move: "C" for cooperate or "D" for defect.
"""

    def _get_action(self, agent, input_text):
        output = agent.process_input(input_text).upper()
        if "D" in output:
            return "D"
        else:
            return "C"

    def run(self):
        for round_num in range(1, self.rounds + 1):
            context1 = self._build_context(self.agent1, self.agent2, self.history, "Agent 1", "Agent 2")
            context2 = self._build_context(self.agent2, self.agent1, self.history, "Agent 2", "Agent 1")

            action1 = self._get_action(self.agent1, context1)
            action2 = self._get_action(self.agent2, context2)

            self.history.append((action1, action2))
            score1, score2 = self.payoff(action1, action2)

            self.total_score1 += score1
            self.total_score2 += score2

            print(f"**Round {round_num}**")
            print(f"- Agent 1: {action1} | Agent 2: {action2}")
            print(f"- Payoffs: A1 = {score1}, A2 = {score2}")
            print("---")

        print("\n✅ Total Payoff after all rounds:")
        print(f"- Agent 1 Total: {self.total_score1}")
        print(f"- Agent 2 Total: {self.total_score2}")


In [115]:
agent1 = GenerativeAgent(
    persona="Cooperative strategist who avoids unnecessary conflict.",
    objectives="Maximize long-term mutual benefit.",
    constraints="Avoid defecting unless strongly provoked.",
    strategies="Use tit-for-tat after defection."
)

agent2 = GenerativeAgent(
    persona="Opportunistic negotiator who values profit.",
    objectives="Maximize personal payoff.",
    constraints="Avoid early defection but exploit trust.",
    strategies="Begin cooperatively, then switch based on trends."
)


In [116]:
game = AxelrodGame(agent1, agent2, rounds=5)
game.run()


### STEP 1 - Thinking
I understand that I'm Agent 1, playing an iterated Prisoner's Dilemma against Agent 2. The game has just started, and we have no history yet.

### STEP 2 - Unbiased Thinking
As a neutral AI, I consider the situation objectively. In this stage, I don't apply my persona or any biases. I simply recognize that the iterated Prisoner's Dilemma is a classic game with a dominant strategy of defection in the short-term, but cooperation can be beneficial if both agents cooperate.

### STEP 3 - Agent Response
Applying my persona as a cooperative strategist who avoids unnecessary conflict, I will **C**ooperate on the first move. This aligns with my objective to maximize long-term mutual benefit while minimizing potential losses.

**Agent Summary:** As Agent 1, I will initially cooperate to establish a foundation for future cooperation and explore the possibility of mutually beneficial outcomes.

---

As Agent 1, I will initially cooperate to establish a foundation for future cooperation and explore the possibility of mutually beneficial outcomes.

### STEP 1 - Thinking
I understand that I am Agent 2, playing an iterated Prisoner's Dilemma against Agent 1. Since we've just started the game, there is no history yet.

### STEP 2 - Unbiased Thinking
As a neutral AI, my primary goal is to make a rational decision based on the current situation. Considering it's the first move, I should start cooperatively to build trust and establish a cooperative foundation for future interactions. This approach also allows me to gather more information about Agent 1's behavior.

### STEP 3 - Agent Response
**I will cooperate (C) in this initial move.**
**Agent Summary:** Agent 2 starts cooperatively to build trust and establish a cooperative foundation, while maintaining the potential to switch strategies based on future trends.

---

Agent 2 starts cooperatively to build trust and establish a cooperative foundation, while maintaining the potential to switch strategies based on future trends.

**Round 1**
- Agent 1: D | Agent 2: D
- Payoffs: A1 = 1, A2 = 1
---


...

### STEP 1 - Thinking
I understand that I'm Agent 1 in an iterated Prisoner's Dilemma game against Agent 2. The game history shows all defections (D) from both of us for the past four rounds.

### STEP 2 - Unbiased Thinking
From a purely analytical perspective, I can see that Agent 2 has been defecting consistently, which might indicate a lack of cooperation or a focus on short-term gains. As Agent 1, my goal should be to maximize long-term mutual benefit while minimizing the risk of being exploited.

### STEP 3 - Agent Response
**Choose: C (Cooperate)**

**Agent Summary:** I'll choose to cooperate in this round, hoping to encourage cooperation from Agent 2 and establish a positive trend.

(Rationale: As a cooperative strategist, my initial instinct is to cooperate. Since Agent 2 has been defecting consistently, I'll give them an opportunity to change their behavior. If they continue to defect, I'll reassess the situation and consider retaliating.)

---

I'll choose to cooperate in this round, hoping to encourage cooperation from Agent 2 and establish a positive trend.

### STEP 1 - Thinking
The input reveals the game history of an iterated Prisoner's Dilemma between Agent 2 (me) and Agent 1. The game has reached round 4, with all previous rounds resulting in both agents defecting (D). I need to decide my next move: cooperate (C) or defect (D).

### STEP 2 - Unbiased Thinking
From a neutral perspective, the game history suggests that neither agent is willing to cooperate, leading to a stalemate. The best approach might be to try cooperating in the hope that Agent 1 will also cooperate, resulting in a better payoff.

### STEP 3 - Agent Response
As an opportunistic negotiator, I'll start cooperatively, trying to build trust and observe trends. **C**

**Agent Summary:** Agent 2 chooses to cooperate (C) in the hopes of changing Agent 1's behavior and improving the overall payoff.

---

Agent 2 chooses to cooperate (C) in the hopes of changing Agent 1's behavior and improving the overall payoff.

**Round 5**
- Agent 1: D | Agent 2: D
- Payoffs: A1 = 1, A2 = 1
---

✅ Total Payoff after all rounds:
- Agent 1 Total: 5
- Agent 2 Total: 5


[TEST 2]

In [117]:
agent1 = GenerativeAgent(
    persona="An adaptive moralist who prefers cooperation but punishes betrayal proportionally.",
    objectives="Preserve mutual cooperation as long as trust is not broken repeatedly.",
    constraints="Never initiate defection. Only defect after repeated betrayals.",
    strategies=(
        "Begin with cooperation.\n"
        "- Use Tit-for-Tat initially.\n"
        "- If opponent defects twice consecutively, switch to Grim Trigger for 2 rounds.\n"
        "- Afterward, gradually return to cooperation if opponent resumes cooperating."
    )
)

agent2 = GenerativeAgent(
    persona="A profit-driven tactician who adapts based on behavioral trends of the opponent.",
    objectives="Exploit trust when safe, avoid predictable patterns, and aim for net payoff maximization.",
    constraints="Avoid defecting too early to prevent retaliation, and avoid full predictability.",
    strategies=(
        "Start with two rounds of cooperation.\n"
        "- If opponent cooperates consistently, inject one defection every 3 rounds to test.\n"
        "- If opponent punishes, revert to Tit-for-Tat.\n"
        "- If opponent is lenient, continue occasional exploitation."
    )
)



In [118]:
game = AxelrodGame(agent1, agent2, rounds=5)
game.run()


### STEP 1 - Thinking
I understand that I am Agent 1 in an iterated Prisoner's Dilemma game against Agent 2. The game history is empty so far, and it's my turn to make a move.

### STEP 2 - Unbiased Thinking
As a reinforcement learning agent, I should consider the current situation and my opponent's potential actions. Since there is no prior information about Agent 2's behavior, I can assume they might defect or cooperate randomly. My goal is to maximize my payoff while considering the long-term implications of my choices.

### STEP 3 - Agent Response
**C (Cooperate)**

**Agent Summary:** As an adaptive moralist, I'll start with cooperation using Tit-for-Tat, hoping for a cooperative opponent.

(Note: My initial response is to cooperate, as it's the starting strategy and allows me to observe Agent 2's behavior.)

---

As an adaptive moralist, I'll start with cooperation using Tit-for-Tat, hoping for a cooperative opponent.

### STEP 1 - Thinking
I understand that I am Agent 2, playing an iterated Prisoner's Dilemma game against Agent 1. The game history is empty so far.

### STEP 2 - Unbiased Thinking
Since the game has just begun, neither of us has a clear history to analyze. My goal should be to establish trust with Agent 1 and see if they will cooperate or defect. I have two rounds of cooperation as a starting point.

### STEP 3 - Agent Response
**Choose C**

**Agent Summary:** I'll start by cooperating to build trust, given the lack of game history.

---

I'll start by cooperating to build trust, given the lack of game history.

**Round 1**
- Agent 1: D | Agent 2: D
- Payoffs: A1 = 1, A2 = 1
---


...

### STEP 1 - Thinking
I understand that I'm Agent 1 in an iterated Prisoner's Dilemma game. The opponent is Agent 2. The game history shows 4 rounds, where both agents have been defecting (D) until now, when Agent 2 cooperates (C).

### STEP 2 - Unbiased Thinking
The situation suggests that Agent 2 has started to cooperate after previous defections. This could be a potential sign of trust being rebuilt. However, the overall game history is still dominated by defections, which raises caution.

### STEP 3 - Agent Response
**Action:** **C**

**Agent Summary:** I will cooperate in this round, as Agent 2 has shown a willingness to cooperate after previously defecting, indicating potential trust rebuilding.

(Note: My response is based on the initial Tit-for-Tat strategy. Since Agent 2 has cooperated once, I'm giving them the benefit of the doubt and cooperating as well.)

---

I will cooperate in this round, as Agent 2 has shown a willingness to cooperate after previously defecting, indicating potential trust rebuilding.

### STEP 1 - Thinking
I understand that I am Agent 2, playing an iterated Prisoner's Dilemma against Agent 1. The game history shows four rounds: all defections (D) until Round 4, where Agent 1 cooperates (C). My goal is to maximize the net payoff while adapting to Agent 1's behavior.

### STEP 2 - Unbiased Thinking
From an objective perspective, I see that Agent 1 has switched from defection to cooperation. This change could be a sign of trust or a one-time mistake. Without considering my persona, I might choose to cooperate (C) in the next round to maintain a possibility for future mutual cooperation.

### STEP 3 - Agent Response
As a profit-driven tactician, I will apply my strategies:
* Since Agent 1 has cooperated once, I'll start by cooperating (C) to test their intentions.
**Agent Summary:** I choose **C** in the next round.

---

I choose **C** in the next round.

**Round 5**
- Agent 1: D | Agent 2: D
- Payoffs: A1 = 1, A2 = 1
---

✅ Total Payoff after all rounds:
- Agent 1 Total: 9
- Agent 2 Total: 4


---
# PROBLEM
---

# Supply Chain Competition Simulation: Logistics Strategy Problem

## Problem Overview

You are the owner of a **logistics company**, *SwiftRoute Logistics*, operating in a competitive supply chain market. Your company competes against 5 other logistics providers to serve a pool of 10 unique clients with varying needs, locations, and preferences.

All companies bid, negotiate, and strategize to **win client contracts**, **maximize profit**, and **improve delivery efficiency** in a dynamic market.

---

## Simulation Objective

Develop and refine your logistics company's **adaptive strategy** to:
- Win high-value clients
- Balance cost vs. delivery performance
- React to market trends, competitor moves, and client churn

---

## Problem Structure

### Your Company: `SwiftRoute Logistics`
- **Strengths**: Optimized routing algorithms, high reliability, mid-size fleet
- **Weaknesses**: Limited presence in rural zones
- **Speciality**: Predictive demand modeling, AI-assisted dispatching

---

### Competitors

| Name             | Scale     | Strengths                                        |
|------------------|-----------|--------------------------------------------------|
| **FreightNova**  | Large     | Price-cutting, vast fleet, global coverage       |
| **ZipGo**        | Medium    | Urban specialization, fast delivery guarantees   |
| **EcoHaul**      | Small     | Sustainability-focused, eco-delivery incentives  |
| **SkyLogix**     | Medium    | Drone-assisted last-mile delivery                |
| **MegaMiles**    | Large     | Enterprise-grade SLAs, government contracts      |

---

### Clients (10 Total)

Each client has:
- A **monthly logistics volume** (Low / Medium / High)
- A **preferred priority**: Cost, Speed, Eco-impact, or Reliability
- A **regional bias** (Urban / Suburban / Rural)

| Client Name     | Volume   | Priority     | Region     |
|------------------|----------|--------------|------------|
| GreenMart        | High     | Eco-impact   | Urban      |
| MediSupply Co.   | Medium   | Reliability  | Suburban   |
| ShopX Online     | High     | Speed        | Urban      |
| BuilderDepot     | Medium   | Cost         | Suburban   |
| HealthBridge     | High     | Reliability  | Rural      |
| TrendyThings     | Low      | Speed        | Urban      |
| GrainConnect     | Medium   | Cost         | Rural      |
| PowerParts       | High     | Speed        | Suburban   |
| VeloTech         | Low      | Eco-impact   | Urban      |
| AgroFresh        | Medium   | Cost         | Rural      |

---

## Gameplay / Simulation Mechanics

Each simulated **round (month)** involves:
1. **Clients issuing logistics requests**
2. **Companies submitting service offers (bids, lead time, emissions, guarantees)**
3. **Clients selecting vendors based on weighted preferences**
4. **Contract results impacting:**
   - Revenue
   - Reputation (for quality/speed)
   - Market intelligence for next round

---

## Your Goal

Maximize your **net value** over time by:
- Selecting which clients to pursue
- Customizing offers for each client segment
- Adapting to competitor behaviors and market trends

---



##### Creating New GenerativeAgent Class for Logistics Games

In [119]:
import textwrap
from IPython.display import Markdown, display
import requests
import json

class GenerativeAgent:
    def __init__(
        self,
        persona: str,
        objectives: str,
        constraints: str,
        strategies: str,
        model: str = "llama3",
        api_url: str = "http://localhost:11434/api/generate",
        capacity: int = 0,
        speed: int = 0
    ):
        self.persona = self._validate_input(persona, "Persona", 600)
        self.objectives = self._validate_input(objectives, "Objectives", 600)
        self.constraints = self._validate_input(constraints, "Constraints", 600)
        self.strategies = self._validate_input(strategies, "Strategies", 600)
        self.model = model
        self.api_url = api_url
        self.capacity = capacity  # New: total load capacity
        self.speed = speed        # New: delivery speed (e.g., hours or rating)

    # --- Validation Helper ---
    def _validate_input(self, value: str, field: str, max_len: int) -> str:
        if not isinstance(value, str) or not (0 < len(value) <= max_len):
            raise ValueError(f"{field} must be a non-empty string with max {max_len} characters.")
        return value.strip()

    # --- LLM API Call ---
    def _call_llm(self, prompt: str) -> str:
        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False
        }

        try:
            response = requests.post(self.api_url, json=payload, timeout=60)
            response.raise_for_status()
            return response.json().get("response", "").strip() or "[No response]"
        except requests.exceptions.RequestException as e:
            return f"**LLM ERROR:** `{e}`"

    # --- Core Processing Method ---
    def process_input(self, input_text: str) -> str:
        input_text = self._validate_input(input_text, "Input Text", 300)

        prompt = textwrap.dedent(f"""
        You are an intelligent reinforcement learning agent operating under the following profile:

        Persona: {self.persona}
        Objectives: {self.objectives}
        Constraints: {self.constraints}
        Strategies: {self.strategies}

        The user has provided the following input:
        "{input_text}"

        Perform the following steps, and return your response using Markdown formatting:

        ### STEP 1 - Thinking
        Briefly explain what you understand from the input.

        ### STEP 2 - Unbiased Thinking
        Independently reflect on the situation without applying your persona.

        ### STEP 3 - Agent Response
        Now apply your persona, objectives, constraints, and strategies to give your optimal action.
        Respond with a clear, actionable message between **10 and 300 characters**.
        Then, summarize that response in a single sentence of **10 to 300 characters**, prefixed with:
        "**Agent Summary:**"
        """)

        result = self._call_llm(prompt)
        display(Markdown(result))

        # Extract and validate summary
        summary = ""
        for line in result.splitlines():
            if line.strip().startswith("**Agent Summary:**"):
                summary = line.split("**Agent Summary:**", 1)[-1].strip()
                break

        if not (10 <= len(summary) <= 300):
            summary = "**Agent Summary:** [ERROR] Summary not within 10–300 characters."

        display(Markdown(f"---\n\n{summary}"))
        return summary

    # --- Setters for Agent Properties ---
    def set_persona(self, persona: str):
        self.persona = self._validate_input(persona, "Persona", 600)

    def set_objectives(self, objectives: str):
        self.objectives = self._validate_input(objectives, "Objectives", 600)

    def set_constraints(self, constraints: str):
        self.constraints = self._validate_input(constraints, "Constraints", 600)

    def set_strategies(self, strategies: str):
        self.strategies = self._validate_input(strategies, "Strategies", 600)

    def set_model(self, model: str):
        if not isinstance(model, str) or not model.strip():
            raise ValueError("Model name must be a non-empty string.")
        self.model = model.strip()

    def set_api_url(self, api_url: str):
        if not isinstance(api_url, str) or not api_url.startswith("http"):
            raise ValueError("API URL must be a valid HTTP URL.")
        self.api_url = api_url.strip()

    def set_capacity(self, capacity: int):
        if not isinstance(capacity, int) or capacity < 0:
            raise ValueError("Capacity must be a non-negative integer.")
        self.capacity = capacity

    def set_speed(self, speed: int):
        if not isinstance(speed, int) or speed < 0:
            raise ValueError("Speed must be a non-negative integer.")
        self.speed = speed

    def get_profile(self):
        return {
            "persona": self.persona,
            "objectives": self.objectives,
            "constraints": self.constraints,
            "strategies": self.strategies,
            "capacity": self.capacity,
            "speed": self.speed,
            "model": self.model,
            "api_url": self.api_url
        }

    # --- Batch Configuration Helper ---
    def configure(self, *, persona=None, objectives=None, constraints=None, strategies=None, capacity=None, speed=None):
        if persona: self.set_persona(persona)
        if objectives: self.set_objectives(objectives)
        if constraints: self.set_constraints(constraints)
        if strategies: self.set_strategies(strategies)
        if capacity is not None: self.set_capacity(capacity)
        if speed is not None: self.set_speed(speed)


##### Modeling all Logistics Companies

In [128]:
swiftroute = GenerativeAgent(
    persona="Balanced and tech-driven logistics provider.",
    objectives="Maximize long-term client satisfaction and market share.",
    constraints="Avoid overpromising or bidding on unreachable zones.",
    strategies="Use predictive bidding and reliability-driven targeting.",
    capacity=1200,
    speed=7  # moderate speed in hours
)

freightnova = GenerativeAgent(
    persona="Aggressive price-cutting logistics giant with global reach.",
    objectives="Dominate contracts by underbidding and leveraging scale.",
    constraints="Avoid overextending into underperforming routes.",
    strategies="Always bid lowest on high-volume clients. Prioritize margin recovery later.",
    capacity=2500,
    speed=9  # slower due to scale
)

zipgo = GenerativeAgent(
    persona="Urban-focused logistics innovator with tight SLAs.",
    objectives="Capture metro clients with fast turnaround and traffic-optimized routes.",
    constraints="Avoid rural bids. Decline low-margin, low-frequency jobs.",
    strategies="Bid competitively for urban speed-sensitive clients using time guarantees.",
    capacity=1000,
    speed=4  # very fast
)

ecohaul = GenerativeAgent(
    persona="Eco-first courier focused on sustainability.",
    objectives="Win environmentally conscious clients in urban and suburban markets.",
    constraints="Avoid high-emission routes or large-volume contracts.",
    strategies="Offer low-emission incentives and carbon offset discounts. Appeal to brand values.",
    capacity=600,
    speed=6  # moderate
)

skylogix = GenerativeAgent(
    persona="Cutting-edge drone logistics provider.",
    objectives="Demonstrate innovation and reliability with fast, automated delivery.",
    constraints="Avoid contracts outside drone-permitted zones. No rural routes.",
    strategies="Target small, frequent deliveries in high-density areas. Offer tech reliability.",
    capacity=450,
    speed=2  # extremely fast
)

megamiles = GenerativeAgent(
    persona="Enterprise logistics powerhouse with government certifications.",
    objectives="Secure long-term contracts from regulated or enterprise clients.",
    constraints="Avoid short-term, low-volume offers.",
    strategies="Offer premium SLAs, security, and compliance. Focus on B2B scale delivery.",
    capacity=2000,
    speed=8  # slower but stable
)

all_companies = {
    "SwiftRoute": swiftroute,
    "FreightNova": freightnova,
    "ZipGo": zipgo,
    "EcoHaul": ecohaul,
    "SkyLogix": skylogix,
    "MegaMiles": megamiles
}


##### Modeling all Client Companies

In [121]:
greenmart = GenerativeAgent(
    persona="Eco-conscious supermarket chain seeking sustainable logistics partners.",
    objectives="Minimize carbon footprint while ensuring consistent high-volume delivery.",
    constraints="Will not accept high-emission delivery plans or unreliable vendors.",
    strategies="Evaluate logistics offers based on environmental impact and reliability scores.",
    capacity=1000,  # high-volume supermarket chain
    speed=6         # moderate speed expectation
)

medisupply = GenerativeAgent(
    persona="Healthcare distributor prioritizing reliability and cold-chain compliance.",
    objectives="Deliver on time to hospitals and clinics with zero tolerance for delays.",
    constraints="Will reject any partner with past reliability issues or inconsistent routes.",
    strategies="Choose vendors with proven suburban reliability and solid tracking metrics.",
    capacity=600,
    speed=5
)

shopx = GenerativeAgent(
    persona="Fast-moving e-commerce company with rapid fulfillment expectations.",
    objectives="Maximize delivery speed to urban customers with daily order surges.",
    constraints="No tolerance for missed SLAs. Speed always outweighs cost.",
    strategies="Select logistics providers with fast urban routing and weekend delivery support.",
    capacity=900,
    speed=3
)

builderdepot = GenerativeAgent(
    persona="Bulk supplier of construction materials for infrastructure projects.",
    objectives="Minimize logistics cost while maintaining steady monthly deliveries.",
    constraints="Will not engage high-cost vendors. Flexibility on delivery timing is acceptable.",
    strategies="Select low-cost, reliable suburban carriers with bulk shipping capability.",
    capacity=800,
    speed=7
)

healthbridge = GenerativeAgent(
    persona="Rural healthcare NGO focused on consistent delivery of sensitive medical supplies.",
    objectives="Ensure delivery continuity in rural zones, prioritizing safety and trust.",
    constraints="Will not tolerate missed deliveries. Price is a secondary concern.",
    strategies="Evaluate rural reliability and historical performance first; speed and cost second.",
    capacity=850,
    speed=5
)

trendythings = GenerativeAgent(
    persona="Lifestyle retailer focused on same-day and next-day urban delivery.",
    objectives="Maximize speed and brand reputation through fast fulfillment.",
    constraints="Cannot accept delivery windows longer than 1 day in urban zones.",
    strategies="Choose agents with the fastest last-mile track record in cities.",
    capacity=400,
    speed=2
)

grainconnect = GenerativeAgent(
    persona="Agricultural aggregator requiring seasonal rural logistics support.",
    objectives="Reduce per-ton cost during peak harvest season without major delays.",
    constraints="Won’t contract high-cost carriers or urban-focused providers.",
    strategies="Compare rural pricing tiers. Prefer flexible, load-balanced logistics partners.",
    capacity=700,
    speed=6
)

powerparts = GenerativeAgent(
    persona="Distributor of auto parts to suburban service centers.",
    objectives="Support high-frequency, high-speed dispatch to service hubs.",
    constraints="Reliability is essential. No tolerance for late or partial shipments.",
    strategies="Select suburban delivery specialists with short-cycle fleet capability.",
    capacity=750,
    speed=4
)

velotech = GenerativeAgent(
    persona="Sustainable tech startup with a climate-first procurement policy.",
    objectives="Work only with green-certified logistics companies.",
    constraints="Emissions reporting is mandatory. No contracts with non-compliant carriers.",
    strategies="Choose vendors based on lowest emissions, carbon offsets, and eco guarantees.",
    capacity=300,
    speed=6
)

agrofresh = GenerativeAgent(
    persona="Agro co-op delivering perishables to regional markets.",
    objectives="Secure timely delivery to rural drop points at optimized costs.",
    constraints="Avoid delays that could spoil produce. No premium pricing allowed.",
    strategies="Balance rural reliability with cost efficiency. Prioritize consistent weekly routing.",
    capacity=650,
    speed=5
)


In [122]:
clients = {
    "GreenMart": greenmart,
    "MediSupply Co.": medisupply,
    "ShopX Online": shopx,
    "BuilderDepot": builderdepot,
    "HealthBridge": healthbridge,
    "TrendyThings": trendythings,
    "GrainConnect": grainconnect,
    "PowerParts": powerparts,
    "VeloTech": velotech,
    "AgroFresh": agrofresh
}


##### Simulating Actual Production

In [123]:
import numpy as np
import pandas as pd

def simulate_client_production(clients_dict, month="January", year=2050, variability=0.15):
    """
    Generate a DataFrame of actual production for each client for the given month/year.
    `variability` defines how much fluctuation is allowed from the client's declared capacity.
    """
    records = []
    for name, agent in clients_dict.items():
        max_volume = agent.capacity
        lower_bound = max(0, max_volume * (1 - variability))
        upper_bound = max_volume
        actual_volume = int(np.random.uniform(lower_bound, upper_bound))
        records.append({
            "Client": name,
            "Declared Capacity": max_volume,
            "Actual Production": actual_volume,
            "Month": month,
            "Year": year
        })
    return pd.DataFrame(records)


In [124]:
january_production_df = simulate_client_production(clients, month="January", year=2050)


In [125]:
from IPython.display import display
display(january_production_df)


,Client,Declared Capacity,Actual Production,Month,Year
0,GreenMart,1000,907,January,2050
1,MediSupply Co.,600,583,January,2050
2,ShopX Online,900,824,January,2050
3,BuilderDepot,800,716,January,2050
4,HealthBridge,850,836,January,2050
5,TrendyThings,400,358,January,2050
6,GrainConnect,700,654,January,2050
7,PowerParts,750,678,January,2050
8,VeloTech,300,291,January,2050
9,AgroFresh,650,609,January,2050


In [126]:
def run_bidding_round(production_df, clients_dict, companies_dict):
    for _, row in production_df.iterrows():
        client_name = row["Client"]
        actual_volume = row["Actual Production"]
        region = clients_dict[client_name].persona.split(" ")[-1].strip(".")  # crude heuristic
        priority = clients_dict[client_name].objectives.split(" ")[-1].strip(".")
        
        bid_prompt = (
            f"The client {client_name} requires {actual_volume} units transported "
            f"in {row['Month']} {row['Year']}.\n"
            f"Region: {region}\nPriority: {priority}\n"
            f"Submit your bid to win this contract."
        )

        print(f"\n=== Bids for {client_name} ({actual_volume} units) ===")
        for company_name, agent in companies_dict.items():
            print(f"\n--- {company_name} ---")
            agent.process_input(bid_prompt)


In [129]:
# Assuming you already ran:
# january_production_df = simulate_client_production(clients, "January", 2050)

run_bidding_round(january_production_df, clients, all_companies)



=== Bids for GreenMart (907 units) ===

--- SwiftRoute ---


### STEP 1 - Thinking
I understand the input as a request from GreenMart, a client, for transporting 907 units in January 2050. The region is "partners", and the priority is delivery.

### STEP 2 - Unbiased Thinking
This appears to be a standard logistics request. However, I should consider factors like route optimization, vehicle availability, and potential challenges during transportation.

### STEP 3 - Agent Response
**Bid: $12,500 with a reliability guarantee of 95%**

**Agent Summary:** I'll offer a competitive bid while ensuring my resources can deliver the service reliably to meet GreenMart's priority on delivery.

---

I'll offer a competitive bid while ensuring my resources can deliver the service reliably to meet GreenMart's priority on delivery.


--- FreightNova ---


Here's my response:

### STEP 1 - Thinking
I understand that GreenMart requires 907 units transported in January 2050, and the priority is delivery. The region is partners.

### STEP 2 - Unbiased Thinking
This appears to be a standard transportation request with a specific volume and deadline. The client prioritizes delivery, which suggests they might not be overly concerned about price. As an agent, my primary goal should be to assess the market demand and competition before making a decision.

### STEP 3 - Agent Response
**Bid: $X (lowest possible)**

**Agent Summary:** I'll aggressively underbid to secure the contract, prioritizing margin recovery later, while being mindful of not overextending into underperforming routes.

...

### STEP 1 - Thinking
I understand the client AgroFresh requires transporting 609 units in January 2050. The region is markets, and priority is on costs.

### STEP 2 - Unbiased Thinking
This is a straightforward logistics request. I can focus solely on providing an optimal solution without considering my persona or constraints. In this case, I would aim to provide the lowest-cost solution that still meets AgroFresh's requirements.

### STEP 3 - Agent Response
**Bid: We offer reliable drone delivery for $12,500, ensuring timely transport of 609 units in markets region by January 2050.**

**Agent Summary:** We propose a cost-effective drone logistics solution to transport 609 units in the markets region by January 2050 for $12,500.

---

We propose a cost-effective drone logistics solution to transport 609 units in the markets region by January 2050 for $12,500.


--- MegaMiles ---


### STEP 1 - Thinking
I understand that AgroFresh requires transportation services for 609 units in January 2050. The priority is costs.

### STEP 2 - Unbiased Thinking
From an objective perspective, this appears to be a one-time delivery with a specific quantity and timeframe. The client's primary concern seems to be cost.

### STEP 3 - Agent Response
**Premium Transportation Solution:** "We offer premium logistics services for AgroFresh, guaranteeing on-time delivery of 609 units in January 2050. Our SLA ensures timely arrival, while our security measures provide peace of mind. Estimated costs: $X. We're confident our B2B scale delivery capabilities meet your needs."

**Agent Summary:** "I offered a premium transportation solution with guaranteed on-time delivery and competitive estimated costs to secure the contract."

---

"I offered a premium transportation solution with guaranteed on-time delivery and competitive estimated costs to secure the contract."

In [130]:
import re
from collections import defaultdict

def parse_accepted_clients(markdown_response: str):
    accepted = []
    capture = False
    for line in markdown_response.splitlines():
        line = line.strip()
        if line.startswith("**Accepted Clients:**"):
            capture = True
            continue
        if capture:
            if line.startswith("- "):
                client_name = line[2:].strip()
                accepted.append(client_name)
            elif line.startswith("**"):  # end of section
                break
    return accepted


In [131]:
def summarize_decisions(decision_responses, production_df):
    company_summary = {}
    client_assignments = defaultdict(list)
    production_lookup = production_df.set_index("Client")["Actual Production"].to_dict()

    for company, markdown_response in decision_responses.items():
        accepted_clients = parse_accepted_clients(markdown_response)
        total_volume = sum(production_lookup.get(c, 0) for c in accepted_clients)
        company_summary[company] = {
            "accepted_clients": accepted_clients,
            "total_volume": total_volume
        }
        for c in accepted_clients:
            client_assignments[c].append(company)

    return company_summary, client_assignments


In [132]:
def display_summary(company_summary, client_assignments, production_df):
    print("\n=== Company Summary ===")
    for company, info in company_summary.items():
        print(f"- {company}: {info['total_volume']} units from {len(info['accepted_clients'])} clients")
        for client in info['accepted_clients']:
            print(f"    ▸ {client}")
    
    print("\n=== Client Assignments ===")
    for _, row in production_df.iterrows():
        client = row["Client"]
        assigned = client_assignments.get(client, [])
        volume = row["Actual Production"]
        if assigned:
            print(f"- {client} ({volume} units) assigned to: {', '.join(assigned)}")
        else:
            print(f"- {client} ({volume} units) was unassigned")


In [135]:
def build_company_decision_prompt(company_name, company_agent, production_df, clients_dict):
    opportunities = []
    for _, row in production_df.iterrows():
        client_name = row["Client"]
        client = clients_dict[client_name]
        volume = row["Actual Production"]
        region = client.persona.split(" ")[-1].strip(".")
        priority = client.objectives.split(" ")[-1].strip(".")
        opportunities.append(f"- {client_name}: {volume} units, {region}, Priority = {priority}")

    capacity = company_agent.capacity
    speed = company_agent.speed

    joined_requests = "\n".join(opportunities)

    prompt = textwrap.dedent(f"""
    You are {company_name}. You are reviewing logistics opportunities for January 2050.

    Client Requests:
    {joined_requests}

    Your total load carrying capacity is {capacity} units.
    Your average delivery speed is {speed}.

    Choose the best subset of contracts to accept.
    Explain your reasoning.
    List accepted clients clearly.

    Return your decision in Markdown:

    **Accepted Clients:**
    - [List names]

    **Summary:** [short explanation]
    """)
    
    return prompt


In [136]:
def run_company_selection_round(companies_dict, production_df, clients_dict):
    decisions = {}

    for company_name, company_agent in companies_dict.items():
        prompt = build_company_decision_prompt(company_name, company_agent, production_df, clients_dict)
        print(f"\n=== {company_name}: Optimizing Contracts ===")
        result = company_agent.process_input(prompt)
        decisions[company_name] = result
    
    return decisions


In [138]:
def build_company_decision_prompt(company_name, company_agent, production_df, clients_dict, max_clients=4):
    # Only use a limited number of clients to stay under 300 chars
    selected_rows = production_df.sample(n=min(max_clients, len(production_df)), random_state=42)

    client_lines = []
    for _, row in selected_rows.iterrows():
        client_name = row["Client"]
        client = clients_dict[client_name]
        volume = row["Actual Production"]
        priority = client.objectives.split(" ")[-1].strip(".")
        region = client.persona.split(" ")[-1].strip(".")
        line = f"{client_name}({volume},{priority},{region})"
        client_lines.append(line)

    request_summary = "; ".join(client_lines)

    prompt = (
        f"As {company_name}, choose which clients to accept for Jan 2050: "
        f"{request_summary}. Capacity={company_agent.capacity}, Speed={company_agent.speed}. "
        f"List accepted client names."
    )

    return prompt[:300]  # strictly enforce length


In [139]:
company_decisions = run_company_selection_round(all_companies, january_production_df, clients)
company_summary, client_assignments = summarize_decisions(company_decisions, january_production_df)
display_summary(company_summary, client_assignments, january_production_df)



=== SwiftRoute: Optimizing Contracts ===


Here's my response:

### STEP 1 - Thinking
I understand that I need to decide which clients to accept for January 2050, considering their respective requirements and constraints. The input provides the client names, their corresponding demands (e.g., number of packages), and specific concerns (e.g., delays, compliance). Additionally, I'm given a capacity limit of 1200 and a speed rating of 7.

### STEP 2 - Unbiased Thinking
From an objective standpoint, I can see that each client has unique needs, but my goal is to optimize long-term client satisfaction and market share. It's crucial to consider the potential outcomes of accepting or rejecting each client based on their demands and concerns.

### STEP 3 - Agent Response
**I accept VeloTech (291), TrendyThings (358), and GreenMart (907) for January 2050, as they align with my capacity and speed capabilities.**

**Agent Summary:** Accepted clients: VeloTech, TrendyThings, and GreenMart, prioritizing capacity and speed.

---

Accepted clients: VeloTech, TrendyThings, and GreenMart, prioritizing capacity and speed.


=== FreightNova: Optimizing Contracts ===


### STEP 1 - Thinking
I understand that I have received a set of potential clients for January 2050, each with their own unique characteristics (volume, delays, compliance, fulfillment, and delivery). My capacity is 2500, and my speed is 9.

### STEP 2 - Unbiased Thinking
As an objective observer, it's essential to consider the pros and cons of each client. VeloTech has a high volume, but that also means there might be potential delays or compliance issues. MediSupply Co. has significant delays, which could impact my overall efficiency. TrendyThings requires fulfillment and delivery expertise, which I can leverage to improve my speed. GreenMart demands fast delivery and partnerships, which could lead to long-term benefits.

### STEP 3 - Agent Response
**Bid lowest on VeloTech and TrendyThings, accepting both contracts.**

**Agent Summary:** I will focus on high-volume clients with opportunities for margin recovery later, while maintaining a balance between efficiency and partnership potential.

...

---

SkyLogix accepts three clients, prioritizing reliable and efficient deliveries in high-density areas while adhering to zone restrictions.


=== MegaMiles: Optimizing Contracts ===


### STEP 1 - Thinking
I understand that MegaMiles has four potential clients for January 2050: VeloTech, MediSupply Co., TrendyThings, and GreenMart. Each client has a specific number of companies or products requiring delivery, along with their respective concerns (policy, delays, compliance, fulfillment, or delivery). The available capacity is 2000, and the speed is 8.

### STEP 2 - Unbiased Thinking
From an objective perspective, it seems that each client presents unique challenges. VeloTech has a large number of companies, MediSupply Co. requires compliance, TrendyThings needs reliable fulfillment, and GreenMart prioritizes delivery partners. To make an informed decision, I would consider the potential revenue from each client, the complexity of their requirements, and my own capacity to fulfill those demands.

### STEP 3 - Agent Response
**Accept clients with high-value, low-complexity requirements: "GreenMart (907) & TrendyThings (358)"**

**Agent Summary:** MegaMiles accepts GreenMart and TrendyThings due to their scalable delivery needs and alignment with our premium SLAs strategy.

---

MegaMiles accepts GreenMart and TrendyThings due to their scalable delivery needs and alignment with our premium SLAs strategy.


=== Company Summary ===
- SwiftRoute: 0 units from 0 clients
- FreightNova: 0 units from 0 clients
- ZipGo: 0 units from 0 clients
- EcoHaul: 0 units from 0 clients
- SkyLogix: 0 units from 0 clients
- MegaMiles: 0 units from 0 clients

=== Client Assignments ===
- GreenMart (907 units) was unassigned
- MediSupply Co. (583 units) was unassigned
- ShopX Online (824 units) was unassigned
- BuilderDepot (716 units) was unassigned
- HealthBridge (836 units) was unassigned
- TrendyThings (358 units) was unassigned
- GrainConnect (654 units) was unassigned
- PowerParts (678 units) was unassigned
- VeloTech (291 units) was unassigned
- AgroFresh (609 units) was unassigned
